# Домашка 5. NN. Часть 3


## Frameworks
1. Проэкспериментриуйте с различными моделями из выбранного фреймворка(recbole или другие рассмотренные фреймворки), выбрать лучшую. (2 балла)
    * Запустил разные, из возможных, выбрал лучшую (`RecVAE`).
3. Пробить скор бейзлайна DSSM модели 0.075. (3 балла):
    * Побили скор: 0.088
4. Обернуть в сервис, который должен  критериям читаемости и воспоизводимости. (5 балла):
    * Вынул предсказания, и запихнул в словарь, для несуществующих людей -- возвращаем все то же популярное:)
   * Это все завернуто в класс `Recbole`, в `dev_models/recbole_model.py`. 

In [18]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole
from recbole.model.general_recommender.recvae import RecVAE

import time

from tqdm import tqdm
from rectools.dataset import Dataset
import torch

from dev_eval import read_kion_dataset
import numpy as np
import pandas as pd
import pickle
import warnings
from pprint import pprint


tqdm.pandas()

In [2]:
warnings.filterwarnings("ignore")

# Data

Подгружаем датасет кион, немного преобразовываем данные (названия колонок).

In [3]:
interactions_df = pd.read_csv("../data/data_original/interactions.csv")
users_df = pd.read_csv("../data/data_original/users.csv")
items_df = pd.read_csv("../data/data_original/items.csv")

In [4]:
interactions_df["t_dat"] = pd.to_datetime(interactions_df["last_watch_dt"], format="%Y-%m-%d")
interactions_df["timestamp"] = interactions_df.t_dat.values.astype(np.int64) // 10**9

In [5]:
df = interactions_df[["user_id", "item_id", "timestamp"]].rename(
    columns={"user_id": "user_id:token", "item_id": "item_id:token", "timestamp": "timestamp:float"}
)

Для пайплайна надо сохранить

In [6]:
! mkdir  -p ../data/recbox_data
df.to_csv("../data/recbox_data/recbox_data.inter", index=False, sep="\t")

# Создаем пайплайн обучения

In [7]:
parameter_dict = {
    "data_path": "../data/",
    "USER_ID_FIELD": "user_id",
    "ITEM_ID_FIELD": "item_id",
    "TIME_FIELD": "timestamp",
    "device": "СPU",
    "user_inter_num_interval": "[40,inf)",
    "item_inter_num_interval": "[40,inf)",
    "load_col": {"inter": ["user_id", "item_id", "timestamp"]},
    "neg_sampling": None,
    "epochs": 10,
    "eval_args": {"split": {"RS": [9, 0, 1]}, "group_by": "user", "order": "TO", "mode": "full"},
    "show_progress": False,
    "log_wandb": False,
}
config = Config(model="MultiVAE", dataset="recbox_data", config_dict=parameter_dict)

# init random seed
init_seed(config["seed"], config["reproducibility"])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.CRITICAL)
logger.addHandler(c_handler)

И подгружаем данные + делим на трэйн-тест

In [8]:
dataset = create_dataset(config)

In [9]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

17 Dec 19:28    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
17 Dec 19:28    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


# Эксперименты с моделями

Я всеми силами пытался уменьшить вывод при обучении, но он все равно сыпит все подряд (почти).

Если что, можно промотать до [следующей ячейки](#cellcheck), там я еще раз все резалты вывожу.

In [12]:
%%time
model_list = ["MultiVAE", "MultiDAE", "MacridVAE", "NeuMF", "RecVAE", "RepeatNet"]
results = dict()
for model_name in model_list:
    print(f"Running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset="recbox_data", config_dict=parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)
    results[model_name] = result
    print("===================================================")

Running MultiVAE...


17 Dec 19:30    INFO  ['/home/toharhymes/anaconda3/envs/p39/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/toharhymes/.local/share/jupyter/runtime/kernel-8a55e824-8e7b-4288-9110-0ef159e5bff6.json']
17 Dec 19:30    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../data/recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}

It took 1.20 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.084), ('mrr@10', 0.1695), ('ndcg@10', 0.0825), ('hit@10', 0.3503), ('precision@10', 0.0467)])}
Running MultiDAE...


17 Dec 19:31    INFO  ['/home/toharhymes/anaconda3/envs/p39/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/toharhymes/.local/share/jupyter/runtime/kernel-8a55e824-8e7b-4288-9110-0ef159e5bff6.json']
17 Dec 19:31    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../data/recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}

It took 1.19 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0829), ('mrr@10', 0.1655), ('ndcg@10', 0.081), ('hit@10', 0.3438), ('precision@10', 0.0459)])}
Running MacridVAE...


...

It took 2.68 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0689), ('mrr@10', 0.117), ('ndcg@10', 0.0604), ('hit@10', 0.301), ('precision@10', 0.038)])}
Running RecVAE...


17 Dec 19:39    INFO  ['/home/toharhymes/anaconda3/envs/p39/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/toharhymes/.local/share/jupyter/runtime/kernel-8a55e824-8e7b-4288-9110-0ef159e5bff6.json']
17 Dec 19:39    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../data/recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}

It took 3.29 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0849), ('mrr@10', 0.1697), ('ndcg@10', 0.0828), ('hit@10', 0.3532), ('precision@10', 0.047)])}
Running RepeatNet...


17 Dec 19:42    INFO  ['/home/toharhymes/anaconda3/envs/p39/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/toharhymes/.local/share/jupyter/runtime/kernel-8a55e824-8e7b-4288-9110-0ef159e5bff6.json']
17 Dec 19:42    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../data/recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}

It took 137.06 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.1364), ('mrr@10', 0.0584), ('ndcg@10', 0.0766), ('hit@10', 0.1364), ('precision@10', 0.0136)])}
CPU times: user 15h 34min 5s, sys: 2h 14min 43s, total: 17h 48min 48s
Wall time: 2h 29min 31s


<a id="cellcheck"></a>

Изучим результаты:

In [14]:
pprint(results)

{'MacridVAE': {'best_valid_result': None,
               'best_valid_score': -inf,
               'test_result': OrderedDict([('recall@10', 0.0835),
                                           ('mrr@10', 0.1574),
                                           ('ndcg@10', 0.0788),
                                           ('hit@10', 0.3499),
                                           ('precision@10', 0.0461)]),
               'valid_score_bigger': True},
 'MultiDAE': {'best_valid_result': None,
              'best_valid_score': -inf,
              'test_result': OrderedDict([('recall@10', 0.0829),
                                          ('mrr@10', 0.1655),
                                          ('ndcg@10', 0.081),
                                          ('hit@10', 0.3438),
                                          ('precision@10', 0.0459)]),
              'valid_score_bigger': True},
 'MultiVAE': {'best_valid_result': None,
              'best_valid_score': -inf,
              'test_

Лучшей моделью по precision оказалась `RecVAE` с `precision@10=0.047`. Получим предсказания для сервиса из нее.

In [23]:
config = Config(model="RecVAE", dataset="recbox_data", config_dict=parameter_dict)
model = RecVAE(config, dataset)
checkpoint = torch.load("saved/RecVAE-Dec-17-2023_19-39-51.pth")
model.load_state_dict(checkpoint["state_dict"])

17 Dec 22:07    WARNING  Max value of user's history interaction records has reached 23.254401942926535% of the total.


<All keys matched successfully>

Подгрузим также словарик с айдишниками

In [58]:
uid_to_user_id = dataset.field2token_id[dataset.uid_field]

In [59]:
def recommend_to_user(user_id, dataset, model):
    if user_id in uid_to_user_id:
        model.eval()
        with torch.no_grad():
            uid_series = dataset.token2id(dataset.uid_field, [user_id])
            index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
            new_inter = dataset[index]
            new_inter = new_inter.to(config["device"])
            new_scores = model.full_sort_predict(new_inter)
            new_scores = new_scores.view(-1, test_data.dataset.item_num)
            new_scores[:, 0] = -np.inf
            recommended_item_indices = torch.topk(new_scores, 10).indices[0].tolist()
            recos = dataset.id2token(dataset.iid_field, [recommended_item_indices]).tolist()[0]
        return recos
    return []

Посмотрим сколько для 1 юзера считается

In [60]:
%%time
recommend_to_user("176549", dataset, model)

CPU times: user 375 ms, sys: 0 ns, total: 375 ms
Wall time: 210 ms


['12173',
 '7626',
 '14317',
 '13865',
 '9728',
 '11310',
 '13018',
 '10942',
 '7571',
 '1785']

Многовато, давайте сделаем словарь, и будем доставать из него...

In [86]:
user_to_recos = {}

In [87]:
for user_id in tqdm(uid_to_user_id):
    if user_id != "[PAD]":
        recos_for_user = recommend_to_user(user_id, dataset, model)
        if recos_for_user:
            user_to_recos.update({user_id: recos_for_user})

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13355/13355 [9:00:04<00:00,  2.43s/it]


(Если что, считается оно минут 50, я просто закрыл ноут, и оно всю ночь не считало)

## Создадим финальную модель

Аналогично двум предыдущим задачам, находится в классе Recbole, в recbole_model.py.

In [88]:
class Recbole:
    def __init__(
        self,
        id_2_reco,
        popular_list,
    ):
        if any(
            arg is None
            for arg in [
                id_2_reco,
                popular_list,
            ]
        ):
            raise ValueError("None of the arguments can be None")

        self.id_2_reco = id_2_reco
        self.popular_list = popular_list

    def recommend(self, user_id, k_recos=10):
        if str(user_id) in self.id_2_reco:
            return self.id_2_reco[str(user_id)]
        else:
            return self.popular_list[:k_recos]

In [89]:
kion_data = read_kion_dataset(fast_check=1)
name_popular = "../models/pickle_data/popular.pickle"
loaded_popular = pickle.load(open(name_popular, "rb"))
data_for_predict = Dataset.construct(kion_data["interactions"].df)
max_k = len(kion_data["items"]["item_id"].unique())
sample_popular_user = data_for_predict.user_id_map.external_ids[0]
popular_list = list(
    loaded_popular.recommend(dataset=data_for_predict, users=[0], k=max_k, filter_viewed=False)["item_id"]
)

In [90]:
recbole_recommender_model = Recbole(user_to_recos, popular_list)

## Save it

In [91]:
name_recbole = "../models/pickle_data/recbole.pickle"
pickle.dump(recbole_recommender_model, open(name_recbole, "wb"))

## Use it

In [92]:
name_recbole = "../models/pickle_data/recbole.pickle"
recbole_loaded = pickle.load(open(name_recbole, "rb"))

In [93]:
%%timeit
recbole_loaded.recommend(176549, 10)

246 ns ± 7.22 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [94]:
recbole_loaded.recommend(322, 10)

['9728',
 '13865',
 '10440',
 '3734',
 '7102',
 '12995',
 '15297',
 '7829',
 '4457',
 '4880']